In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

from polaris.dataset import Dataset, ColumnAnnotation
from polaris.benchmark import SingleTaskBenchmarkSpecification
from polaris.utils.types import HubOwner

from tdc.benchmark_group import admet_group

from polaris.evaluate._metric import Metric

root = pathlib.Path("__file__").absolute().parents[2]
# set to recipe root directory
os.chdir(root)
sys.path.insert(0, str(root))

/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the owner and organization
org = "tdcommons"

owner = HubOwner(slug=org, type="organization")
owner

HubOwner(slug='tdcommons', external_id=None, type='organization')

In [3]:
from tdc import utils

names = utils.retrieve_benchmark_names("ADMET_Group")

In [4]:
tdc_info = pd.read_csv(
    "/mnt/ps/home/CORP/lu.zhu/project/polaris-recipes/org-TDC/admet/tdc_admet_info.csv",
    index_col="Name",
)

In [5]:
metric_dict = {
    "MAE": "mean_absolute_error",
    "AUROC": "roc_auc",
    "Spearman": "spearmanr",
    "AUPRC": "pr_auc",
}

Y_dict = {
    "caco2_wang": "Caco permeability readout.",
    "hia_hou": "Human intestinal absorption readout.",
    "pgp_broccatelli": "Pgp inhibition potency.",
    "bioavailability_ma": "Activity of bioavailability.",
    "lipophilicity_astrazeneca": "Activity of lipophilicity.",
    "solubility_aqsoldb": "Aqeuous solubility.",
    "bbb_martins": "Ability of a drug to penetrate the blood-brain barrier.",
    "ppbr_az": "Plasma protein binding rate.",
    "vdss_lombardo": "The volume of distribution at steady state",
    "cyp2d6_veith": "CYP2D6 inhibition.",
    "cyp3a4_veith": "CYP3A4 inhibition.",
    "cyp2c9_veith": "CYP2C9 inhibition.",
    "cyp2d6_substrate_carbonmangels": "CYP2D6 substrate.",
    "cyp3a4_substrate_carbonmangels": "CYP3A4 substrate.",
    "cyp2c9_substrate_carbonmangels": "CYP2C9 substrate.",
    "half_life_obach": "Half life duration.",
    "clearance_microsome_az": "Microsomal intrinsic clearance.",
    "clearance_hepatocyte_az": "Hepatocytic intrinsic clearance.",
    "herg": "hERG inhibition.",
    "ames": "Drug mutagenicity.",
    "dili": "Drug induced liver injury.",
    "ld50_zhu": "Acute toxicity LD50.",
}

In [6]:
tox = [
    "herg",
    "dili",
    "ames",
    "ld50_zhu",
]
target_col = "Y"
mol_col = "Drug"

In [9]:
for name in names:
    print(name)
    group = admet_group(
        path="/mnt/ps/home/CORP/lu.zhu/project/polaris-recipes/org-TDC/admet/data"
    )
    tdc_benchmark = group.get(name)
    tdc_benchmark["train_val"]["split"] = "train_val"
    tdc_benchmark["test"]["split"] = "test"
    table = pd.concat(
        [tdc_benchmark["train_val"], tdc_benchmark["test"]], ignore_index=True
    )

    annotations = {
        "Drug_ID": ColumnAnnotation(description="Drug identifier."),
        "Drug": ColumnAnnotation(
            description="Molecule SMILES string.",
            modality="molecule",
        ),
        "Y": ColumnAnnotation(description=Y_dict[name]),
        "split": ColumnAnnotation(
            description="Train-val-test split provided by TDC.",
        ),
    }

    dataset_name = tdc_benchmark["name"]
    if name in tox:
        source = "https://tdcommons.ai/single_pred_tasks/tox"
        tags = ["TOX"]
    else:
        source = "https://tdcommons.ai/single_pred_tasks/adme"
        tags = ["ADME"]
    dataset = Dataset(
        table=table[annotations.keys()].copy(),
        name=dataset_name,
        description=Y_dict[name],
        source=tdc_info.loc[name, "Reference"],
        annotations=annotations,
        owner=owner,
        tags=tags,
        license="CC-BY-4.0",
    )
    try:
        dataset.upload_to_hub()
    except:
        pass

    metric = tdc_info.loc[name, "Metric"]
    task = (
        "regression" if tdc_info.loc[name, "Task"] == "Regression" else "classification"
    )
    split = (
        table.query("split == 'train_val'").index.tolist(),
        table.query("split == 'test'").index.tolist(),
    )

    benchmark_name = tdc_benchmark["name"]
    benchmark = SingleTaskBenchmarkSpecification(
        name=benchmark_name,
        dataset=dataset,
        target_cols=target_col,
        target_type=task,
        input_cols=[mol_col],
        split=split,
        metrics=[metric_dict[metric]],
        tags=["ADMET", "singletask"],
        owner=owner,
    )
    benchmark.upload_to_hub()

Found local copy...


caco2_wang
⠋ Uploading dataset...

2024-07-22 15:39:24.252 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


💥 ERROR: Failed to upload dataset. 
⠋ Uploading benchmark...

2024-07-22 15:39:24.634 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


...

✅ SUCCESS: Your benchmark has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/tdcommons/dili
 
ld50_zhu
⠋ Uploading dataset...

2024-07-22 15:39:53.262 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


✅ SUCCESS: Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/tdcommons/ld50_zhu
 
⠋ Uploading benchmark...

2024-07-22 15:39:54.298 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


✅ SUCCESS: Your benchmark has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/tdcommons/ld50_zhu
 


In [11]:
import polaris as po

In [20]:
from polaris.utils.misc import sluggify

In [21]:
for name in names:
    try:
        po.load_dataset(f"tdcommons/{sluggify(name)}")
    except:
        print(f"failed {name}")

⠋ Fetching dataset...

⠏ Fetching dataset... 

2024-07-22 15:46:58.100 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched dataset.
 
⠦ Fetching dataset... 

2024-07-22 15:46:59.495 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


...

✅ SUCCESS: Fetched dataset.
 
⠧ Fetching dataset... 

2024-07-22 15:47:15.781 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched dataset.
 
⠇ Fetching dataset... 

2024-07-22 15:47:16.509 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched dataset.
 


In [23]:
for name in names:
    try:
        po.load_benchmark(f"tdcommons/{name}")
    except:
        print(f"failed {name}")

⠙ Fetching benchmark... 

⠏ Fetching benchmark... 

2024-07-22 15:48:27.663 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.
2024-07-22 15:48:27.665 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched dataset.
 
✅ SUCCESS: Fetched benchmark.
 
⠋ Fetching benchmark... 

2024-07-22 15:48:28.517 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.
2024-07-22 15:48:28.520 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


...

✅ SUCCESS: Fetched dataset.
 
✅ SUCCESS: Fetched benchmark.
 
⠏ Fetching benchmark... 

2024-07-22 15:48:44.982 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.
2024-07-22 15:48:44.985 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched dataset.
 
✅ SUCCESS: Fetched benchmark.
 
⠋ Fetching benchmark... 

2024-07-22 15:48:45.849 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.
2024-07-22 15:48:45.854 | WARNING  | polaris._mixins:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched dataset.
 
✅ SUCCESS: Fetched benchmark.
 
